원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

# 1.5 암호학

이 절에서는 암호학 예제로 문자열을 다루는 법을 설명한다. 해당 암호화 기법, *힐 암호 Hill cipher* 는 *유한체 finite field* 에서의 계산을 포함한다.
거의 모든 현대적인 컴퓨터들은 ASCII 문자 집합으로 기본 문장을 저장한다. ASCII 란 *미국 정보 교환 표준 코드 Americal Standard Code for Information Exchange* 라는 뜻이다. 해당 문자 집합은 한 바이트 안 8개의 비트 중 7개를 사용하여 128 문자를 코드로 표시한다. 첫 32글자는 표시되지 않는 제어문자로 탭, 백스페이스, 줄 끝 표시 등이다. 128번째 글자는 또하나의 표시되지 않는 문자인데 여러분 키보드의 삭제 Delete 키에 대응된다. 이러한 제어 문자열 사이에 95자의 표시되는 문자가 자리하는데, 공백, 10진수, 소문자 26자, 대문자 26자와 문장부호 32자를 포함한다.
다음과 같이 표시되는 문자를 ASCII 순서대로 표시할 수 있다.
우선 3 행 16열로 정수 행렬을 준비하여 표시한다.

In [ ]:
import numpy as np
import pprint

x = np.arange(32, 127 + 1, dtype=np.uint8).reshape((3, 32)).tolist()
pprint.pprint(x, width=160)

그 다음 아래와 같이 문자열로 표시할 수 있다.

In [ ]:
c_list = [''.join([chr(x_int) for x_int in x_row]) for x_row in x]
pprint.pprint(c_list)

마지막 문자 127 은 원래는 표시되지 않지만 여기서는 \x7f 로 16진수 형태로 표시되었다.
``c_list`` 의 첫 문자는 공백이다. 다음 명령문의 결과를 확인하라.

In [ ]:
chr(32)